In [74]:
#https://spark.apache.org/docs/latest/sql-data-sources-csv.html
#https://sparkbyexamples.com/pyspark/pyspark-convert-array-column-to-string-column/
#! /usr/bin/python
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[4]").appName("air_quality_pyspark").getOrCreate()

df = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2018.csv")
df19 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2019.csv")
df20 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2020.csv")
df21 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2021.csv")
df= df.union(df19).union(df20).union(df21)
df.show()
#df = df.withColumn("date_time",to_timestamp(df.date_time,'dd/MM/yyyy HH:mm'))
#df = df.withColumn("date",date_format(df.date_time,'yyyy-MM-dd'))

#df = df.withColumn("pollutant_value",df.pollutant_value.cast(DoubleType()))
#df_aggr_daily = df.groupby(['date','station_code','pollutant_id']).mean("pollutant_value").withColumnRenamed("avg(pollutant_value)","mean_pollutant_value")

#df_aggr_daily.write.option("delimiter", ",").option("header", True).mode('overwrite').csv('gs://air_quality_cyprus/Data_Daily_All') 


+----------------+------------+------------+---------------+
|       date_time|station_code|pollutant_id|pollutant_value|
+----------------+------------+------------+---------------+
| 01/01/2018 0:00|           1|          45|            4.8|
| 01/01/2018 1:00|           1|          45|            5.3|
| 01/01/2018 2:00|           1|          45|            6.1|
| 01/01/2018 3:00|           1|          45|            4.0|
| 01/01/2018 4:00|           1|          45|            3.7|
| 01/01/2018 5:00|           1|          45|            1.2|
| 01/01/2018 6:00|           1|          45|            1.0|
| 01/01/2018 7:00|           1|          45|            1.1|
| 01/01/2018 8:00|           1|          45|            1.9|
| 01/01/2018 9:00|           1|          45|            0.9|
|01/01/2018 10:00|           1|          45|            0.4|
|01/01/2018 11:00|           1|          45|            0.3|
|01/01/2018 12:00|           1|          45|            0.5|
|01/01/2018 13:00|      

In [133]:
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

df = df.withColumn("date_time",to_timestamp(df.date_time,'dd/MM/yyyy HH:mm'))
df = df.withColumn("date",date_format(df.date_time,'yyyy-MM-dd'))
df = df.withColumn("pollutant_value",df.pollutant_value.cast(DoubleType()))
df_aggr_daily=  df.groupby(['date','station_code','pollutant_id']).mean("pollutant_value").withColumnRenamed("avg(pollutant_value)","mean_pollutant_value")

df_aggr_daily.show(truncate=False) 

+----------+------------+------------+--------------------+
|date      |station_code|pollutant_id|mean_pollutant_value|
+----------+------------+------------+--------------------+
|2018-01-29|1           |45          |2.4291666666666667  |
|2018-05-24|1           |45          |0.3041666666666667  |
|2018-08-21|1           |45          |0.325               |
|2018-11-22|3           |45          |null                |
|2018-02-27|8           |45          |0.2916666666666666  |
|2018-03-18|8           |45          |0.16250000000000003 |
|2018-05-09|8           |45          |null                |
|2018-12-08|8           |45          |0.9666666666666668  |
|2018-02-12|9           |45          |0.620833333333333   |
|2018-06-25|9           |45          |0.27499999999999997 |
|2018-09-19|9           |45          |0.29166666666666663 |
|2018-11-08|9           |45          |0.5333333333333333  |
|2018-06-07|15          |45          |0.26249999999999996 |
|2018-12-31|14          |45          |0.

In [132]:
df_stations = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/AirQualityMonitoringStations.csv")
df_stations = df_stations.select(col("station_code"),col("station_name_en")) #,col("latitude"),col("longitude")
df_stations = df_stations.withColumnRenamed('station_code','station_code_pk')
df_stations.show()

+---------------+--------------------+
|station_code_pk|     station_name_en|
+---------------+--------------------+
|              1|Nicosia -Traffic ...|
|              2|Nicosia - Residen...|
|              3|Limassol -Traffic...|
|              5|Larnaka - Traffic...|
|              8|Zygi -Industrial ...|
|              9|EMEP- Ayia Marina...|
|             14|Mari - Industrial...|
|             15|Pafos - Traffic S...|
|             16|Paralimni - Traff...|
+---------------+--------------------+



In [126]:
df_Pollutants = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/PollutantsId.csv")
df_Pollutants = df_Pollutants.select(col("pollutant_id"),col("pollutant_code"),col("pollutant_name_en"),col("Unit_of_measurement_en"))
df_Pollutants = df_Pollutants.withColumnRenamed('pollutant_id','pollutant_id_pk')
# Unit_of_measurement_en:μg/m³
df_Pollutants.show()

+---------------+--------------+--------------------+----------------------+
|pollutant_id_pk|pollutant_code|   pollutant_name_en|Unit_of_measurement_en|
+---------------+--------------+--------------------+----------------------+
|              1|            NO|      Nitrogen Oxide|                 μg/m³|
|              2|           NO2|    Nitrogen Dioxide|                 μg/m³|
|              3|           NOX|     Nitrogen Oxides|                 μg/m³|
|              4|           SO2|      Sulfur Dioxide|                 μg/m³|
|              5|            O3|              Ozone |                 μg/m³|
|              6|            CO|     Carbon Monoxide|                 μg/m³|
|             25|          PM10|Particulate Matte...|                 μg/m³|
|             26|         PM2.5|Particulate Matte...|                 μg/m³|
|             45|          C6H6|             Benzene|                 μg/m³|
+---------------+--------------+--------------------+----------------------+

In [134]:
#Join on aggr data
df_aggr_daily = df_aggr_daily.join(df_stations,df_aggr_daily.station_code == df_stations.station_code_pk, how="inner")
df_aggr_daily = df_aggr_daily.join(df_Pollutants,df_aggr_daily.pollutant_id == df_Pollutants.pollutant_id_pk, how="inner")
df_aggr_daily.show()

+----------+------------+------------+--------------------+---------------+--------------------+---------------+--------------+-----------------+----------------------+
|      date|station_code|pollutant_id|mean_pollutant_value|station_code_pk|     station_name_en|pollutant_id_pk|pollutant_code|pollutant_name_en|Unit_of_measurement_en|
+----------+------------+------------+--------------------+---------------+--------------------+---------------+--------------+-----------------+----------------------+
|2018-01-29|           1|          45|  2.4291666666666667|              1|Nicosia -Traffic ...|             45|          C6H6|          Benzene|                 μg/m³|
|2018-05-24|           1|          45|  0.3041666666666667|              1|Nicosia -Traffic ...|             45|          C6H6|          Benzene|                 μg/m³|
|2018-08-21|           1|          45|               0.325|              1|Nicosia -Traffic ...|             45|          C6H6|          Benzene|          

In [135]:
df_aggr_daily = df_aggr_daily.drop("station_code_pk").drop("pollutant_id_pk")
df_aggr_daily.show()

+----------+------------+------------+--------------------+--------------------+--------------+-----------------+----------------------+
|      date|station_code|pollutant_id|mean_pollutant_value|     station_name_en|pollutant_code|pollutant_name_en|Unit_of_measurement_en|
+----------+------------+------------+--------------------+--------------------+--------------+-----------------+----------------------+
|2018-01-29|           1|          45|  2.4291666666666667|Nicosia -Traffic ...|          C6H6|          Benzene|                 μg/m³|
|2018-05-24|           1|          45|  0.3041666666666667|Nicosia -Traffic ...|          C6H6|          Benzene|                 μg/m³|
|2018-08-21|           1|          45|               0.325|Nicosia -Traffic ...|          C6H6|          Benzene|                 μg/m³|
|2018-11-22|           3|          45|                null|Limassol -Traffic...|          C6H6|          Benzene|                 μg/m³|
|2018-02-27|           8|          45|  0

In [136]:
df_aggr_daily.dtypes

[('date', 'string'),
 ('station_code', 'string'),
 ('pollutant_id', 'string'),
 ('mean_pollutant_value', 'double'),
 ('station_name_en', 'string'),
 ('pollutant_code', 'string'),
 ('pollutant_name_en', 'string'),
 ('Unit_of_measurement_en', 'string')]

In [137]:
#Join on aggr data
df_aggr_daily.write.option("delimiter", ",").option("quote", "\"").option("header", True).mode('overwrite').csv('gs://air_quality_cyprus/Data_Daily_All')

In [49]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
df_aggr_daily.write.parquet("data.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("data.parquet")

AnalysisException: 'path hdfs://cluster-spark-m/user/root/data.parquet already exists.;'

In [51]:
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
station_values = spark.sql("SELECT pollutant_id,station_code, avg(mean_pollutant_value)  FROM parquetFile Group by pollutant_id,station_code")
station_values.show()

+------------+------------+-------------------------+
|pollutant_id|station_code|avg(mean_pollutant_value)|
+------------+------------+-------------------------+
|           4|           1|        2.988424547892828|
|           4|           2|       2.1918487950008325|
|           3|           8|       13.187183251701633|
|           5|           1|         58.9167301327806|
|          45|           8|      0.36437009400526466|
|           2|           8|       10.344052205001752|
|           5|           5|        63.89023775298963|
|           1|           1|       15.883170939337798|
|           2|           3|       30.227018379703765|
|           4|           8|       1.5184367062635398|
|           4|          15|       1.4511682767721918|
|          45|          14|        0.159159962067982|
|           5|          16|        70.14754406535972|
|          45|           5|       0.9853151255101394|
|           1|          15|        3.901546972253486|
|           4|          16| 

In [36]:
df_aggr_daily.dtypes

[('date', 'string'),
 ('station_code', 'string'),
 ('pollutant_id', 'string'),
 ('avg(pollutant_value)', 'double')]

In [52]:
df_aggr_daily.write.option("delimiter", ",").option("header", True).mode('overwrite').csv('gs://air_quality_cyprus/Data_Daily_All') 

In [85]:
#union https://stackoverflow.com/questions/37332434/concatenate-two-pyspark-dataframes
#https://sparkbyexamples.com/pyspark/pyspark-to_date-convert-timestamp-to-date/#:~:text=PySpark%20timestamp%20(%20TimestampType%20)%20consists%20of,to%20date%20on%20DataFrame%20column.
#https://sparkbyexamples.com/pyspark/pyspark-date_format-convert-date-to-string-format/

+----------------+------------+------------+---------------+-------------------+----+
|date_time       |station_code|pollutant_id|pollutant_value|date_type          |hour|
+----------------+------------+------------+---------------+-------------------+----+
|01/01/2018 0:00 |1           |45          |4.8            |2018-01-01 00:00:00|00  |
|01/01/2018 1:00 |1           |45          |5.3            |2018-01-01 01:00:00|01  |
|01/01/2018 2:00 |1           |45          |6.1            |2018-01-01 02:00:00|02  |
|01/01/2018 3:00 |1           |45          |4.0            |2018-01-01 03:00:00|03  |
|01/01/2018 4:00 |1           |45          |3.7            |2018-01-01 04:00:00|04  |
|01/01/2018 5:00 |1           |45          |1.2            |2018-01-01 05:00:00|05  |
|01/01/2018 6:00 |1           |45          |1.0            |2018-01-01 06:00:00|06  |
|01/01/2018 7:00 |1           |45          |1.1            |2018-01-01 07:00:00|07  |
|01/01/2018 8:00 |1           |45          |1.9       

Read 

In [55]:
df_readback = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Daily_All")
df_readback                                                                  

DataFrame[date: string, station_code: string, pollutant_id: string, mean_pollutant_value: string]

In [56]:
df_readback.show()

+----------+------------+------------+--------------------+
|      date|station_code|pollutant_id|mean_pollutant_value|
+----------+------------+------------+--------------------+
|2018-03-11|           1|          45|  0.8571428571428571|
|2018-01-28|           3|          45|   2.895454545454545|
|2018-02-01|           3|          45|                null|
|2018-06-30|           3|          45| 0.40833333333333327|
|2018-08-09|           3|          45|                null|
|2018-10-09|           3|          45|                null|
|2018-12-02|           3|          45|                null|
|2018-12-11|           3|          45|                null|
|2018-01-24|           5|          45|  1.6124999999999998|
|2018-02-01|           9|          45|  0.7041666666666665|
|2018-09-14|          15|          45| 0.25833333333333325|
|2018-10-02|          15|          45| 0.36956521739130443|
|2018-02-03|          14|          45|                null|
|2018-02-22|          14|          45|  

In [57]:
pd_df_aggr_daily  = df_aggr_daily.toPandas()

In [67]:
pd_df_aggr_daily[pd_df_aggr_daily.isna()==True].count()

date                    0
station_code            0
pollutant_id            0
mean_pollutant_value    0
dtype: int64

In [69]:
pd_df_aggr_daily.plot()